# Algoritmia
## POO Avanzada

Se pide la implementación de las funciones que aparecen a continuación. 

En el cuerpo de cada función hay una instrucción "pass", se debe sustituir por la implementación adecuada. 

Para cada clase o función que se pide se proporcionan algunos tests. Las implementaciones deberían superar estos tests.

## Preámbulo

In [1]:
# Importaciones
import unittest
from math import log

## Generador de valores de una recurrencia
Dada una recurrencia, se quiere generar valores de la misma

### Función `generador_recurrencia`

In [2]:
def generador_recurrencia(coeficientes, funcion_adicional, iniciales):
    """
    Generador de valores de acuerdo a una recurrencia:
    F(n) = coeficientes[0]*F(n-1) + coeficientes[1]*F(n-2) + ...
         + funcion_adicional(n)
    Los valores iniciales son F(0) = iniciales[0], F(1) = iniciales[1],...
    Los valores que se generan son F(0), F(1), F(2),...
    Se deben generar los valores de uno en uno, no hay que devolver varios.
    Debe generar valores indefinidamente, no hay que poner límites.
    Aunque sea una recurrencia, los valores *no* deben calcularse recursivamente.
    """

    calculados = iniciales
    yield from calculados

    # paso es n
    paso = len(calculados)

    while True:
        # Inicializacion de el marcador (n-1) de F(n-1)
        resta = 1
        calculado = 0
        # Se opera con todos los coeficientes
        for coef in coeficientes:
            # print(f"[{paso}] Usando: {calculados[len(calculados)-resta]}")
            calculado = calculado + coef * calculados[len(calculados)-resta]
            resta += 1

        calculado = calculado + funcion_adicional(paso)
        calculados.append(calculado)

        # limpiar anteriores, no necesita haber mas calculados que operadores falla
        if len(calculados)-1 > len(coeficientes):
            calculados.pop(0)


        yield calculado
        paso += 1
    pass

gen_rec = iter(generador_recurrencia([1], lambda n: n, [0]))
for i in range(0,10):
    print(next(gen_rec))

0
1
3
6
10
15
21
28
36
45


### Tests para `generador_recurrencia`

In [3]:
class TestGeneradorRecurrencia(unittest.TestCase):
    
    @staticmethod
    def comprueba_recurrencia(coeficientes, funcion_adicional, iniciales,
                              funcion_alternativa, numero_comprobaciones=100, 
                              epsilon=0.1):
        """
        Dada una recurrencia (definida en términos de sus coeficientes,
        condiciones inciales y la función_adicional) comprueba si los valores
        generados son (aproximadamente) los mismos que los definidos por una función
        alternativa, para un determinado número de comprobaciones.
        """
    
        iterador = generador_recurrencia(coeficientes, funcion_adicional, 
                                         iniciales)
        for n in range(numero_comprobaciones):
            esperado = next(iterador)
            alternativo = funcion_alternativa(n)
            # print(f"Se obtuvo {esperado} pero se esperaba {alternativo}")
            if abs(esperado - alternativo) > epsilon:
                return False
        return True
    
    """
    Nomenclatura de los tests: test_X_Y_Z.
        X representa los coeficientes. Usamos "n" para valores negativos.
        Y representa la función. Usamos "d" para la división, "p" para la 
        potencia.
        Z representa las condiciones inciales.
    """
          
    def test_1_1_0(self):
        # Recurrencia f(0)=0, f(n)=f(n-1)+1, que se corresponde con f(n)=n
        self.assertTrue(self.comprueba_recurrencia([1], lambda n: 1, [0], lambda n: n))
        
    def test_2_0_1(self):        
        # Recurrencia f(0)=1, f(n)=2*f(n-1), que se corresponde con f(n)=2**n
        self.assertTrue(self.comprueba_recurrencia([2], lambda n: 0, [1], lambda n: 2**n))

    def test_1_n_0(self):        
        # Recurrencia f(0)=0, f(n)=f(n-1)+n, que se corresponde con 
        # f(n)=n*(n+1)/2
        self.assertTrue(self.comprueba_recurrencia([1], lambda n: n, [0],
                                                   lambda n: n * (n + 1) / 2))
        
    def test_1_nd2_0(self):        
        # Recurrencia f(0)=0, f(n)=f(n-1)+n/2, que se corresponde con
        # f(n)=n*(n+1)/4
        self.assertTrue(self.comprueba_recurrencia([1], lambda n: n / 2, [0], 
                                                   lambda n: n * (n + 1) / 4))
        
    def test_1_2pn_1(self):        
        # Recurrencia f(0)=1, f(n)=f(n-1)+2**n, que se corresponde con
        # f(n)=2**(n+1)-1
        self.assertTrue(self.comprueba_recurrencia([1], lambda n: 2 ** n, [1],
                                                   lambda n: 2 ** (n + 1) - 1))

    def test_4n4_0_01(self):        
        # Recurrencia f(0)=0, f(1)=1, f(n)=4f(n-1)-4f(n-2), que se corresponde
        # con f(n)=2**(n-1)*n
        self.assertTrue(self.comprueba_recurrencia([4, -4], lambda n: 0, [0, 1],
                                                   lambda n: 2 ** (n - 1) * n))

    def test_2n1_1_01(self):        
        # Recurrencia f(0)=0, f(1)=1, f(n)=2f(n-1)-f(n-2)+1, que se corresponde
        # con f(n)=n*(n+1)/2
        self.assertTrue(self.comprueba_recurrencia([2, -1], lambda n: 1, [0, 1], 
                                                   lambda n: n * (n + 1) / 2))
        
    def test_11n1_0_012(self):        
        # Recurrencia f(0)=0, f(1)=1, f(2)=2, f(n)=f(n-1)+f(n-2)-f(n-3), que se
        # corresponde con f(n)=n
        self.assertTrue(self.comprueba_recurrencia([1, 1, -1], 
                                                   lambda n: 0, [0, 1, 2], 
                                                   lambda n: n))    

In [4]:
test1 = TestGeneradorRecurrencia()
test1.test_1_n_0()

## Recurrencias del teorema maestro
Se trabaja con recurrencias del tipo que se consideran en el teorema maestro.


### Clase `RecurrenciaMaestra`

In [5]:
class RecurrenciaMaestra: 
    """
    Clase que representa una recurrencia de las que se consideran en el 
    teorema maestro, de la forma T(n)=aT(n/b)+n^k. Se interpreta que en n/b
    la división es entera.
    Además de los métodos que aparecen a continuación, tienen que funcionar 
    los siguientes operadores: 
        ==, !=,
        str(): la representación como cadena debe ser 'aT(n/b)+n^k'
        []: el parámetro entre corchetes es el valor de n para calcular T(n).
    """

    def __init__(self, a, b, k, inicial = 0):
        """
        Constructor de la clase, los parámetros a, b, y k son los que
        aparecen en la fórmula aT(n/b)+n^k. El parámetro inicial es el valor
        para T(0).
        """
        self.__a = a
        self.__b = b
        self.__k = k
        self.__inicial = inicial
        pass

    def metodo_maestro(self):
        """
        Devuelve una cadena con el tiempo de la recurrencia de acuerdo al 
        método maestro. La salida está en el formato "O(n^x)" o "O(n^x*log(n))",
        siendo x un número.
        """
        sol = ""
        if (self.__a > self.__b**self.__k):
            sol = f"θ(n^x*log({self.__a}))"
        elif (self.__a == self.__b**self.__k):
            sol = f"θ()"
        return sol

        pass

    def __iter__(self):
        """
        Generador de valores de la recurrencia: T(0), T(1), T(2), T(3)..., 
        indefinidamente.
        Aunque sea una recurrencia, los valores *no* deben calcularse 
        recursivamente.
        """
        calculados = iniciales
        yield from calculados

        # paso es n
        paso = len(calculados)

        while True:
            # Inicializacion de el marcador (n-1) de F(n-1)
            resta = 1
            calculado = 0
            # Se opera con todos los coeficientes
            for coef in coeficientes:
                # print(f"[{paso}] Usando: {calculados[len(calculados)-resta]}")
                calculado = calculado + coef * calculados[len(calculados)-resta]
                resta += 1

            calculado = calculado + funcion_adicional(paso)
            calculados.append(calculado)

            # limpiar anteriores, no necesita haber mas calculados que operadores falla
            if len(calculados)-1 > len(coeficientes):
                calculados.pop(0)


            yield calculado
            paso += 1
        pass

### Tests para `RecurrenciaMaestra`

In [6]:
class TestRecurrenciaMaestra(unittest.TestCase):
           
    def test_teorema_3_2_2(self):
        # Recurrencia T(n)=3T(n/2)+O(n^2)
        resultado = RecurrenciaMaestra(3, 2, 2).metodo_maestro()
        self.assertEqual(resultado, "O(n^2)")

    def test_teorema_2_2_1(self):
        # Recurrencia T(n)=2T(n/2)+O(n)
        resultado = RecurrenciaMaestra(2, 2, 1).metodo_maestro()
        self.assertEqual(resultado, "O(n^1*log(n))")

    def test_teorema_3_2_1(self):
        # Recurrencia T(n)=3T(n/2)+O(n)
        resultado = RecurrenciaMaestra(3, 2, 1).metodo_maestro()
        # esperamos algo parecido a "O(n^1.5849625007211563)"
        self.assertTrue("O(n^1.58" in resultado)
        self.assertTrue("log" not in resultado)
        
    def test_operador_eq(self):
        # Tests para los operadores == y !=

        r = RecurrenciaMaestra(2, 2, 2)

        self.assertTrue(r == RecurrenciaMaestra(2, 2, 2))
        self.assertFalse(r != RecurrenciaMaestra(2, 2, 2))
        for a, b, k in ((1, 1, 1), (1, 1, 2), (1, 2, 1), (2, 1, 1) ):
            self.assertTrue(r != RecurrenciaMaestra(a, b, k))
            self.assertFalse(r == RecurrenciaMaestra(a, b, k))

    def test_operador_str(self):
        # Tests para str()
        self.assertEqual(str(RecurrenciaMaestra(2, 2, 2)), "2T(n/2)+n^2")
        self.assertEqual(str(RecurrenciaMaestra(7, 4, 3)), "7T(n/4)+n^3")

    # Tests para []        
    
    def test_operador_getitem_222(self):        
       
        r = RecurrenciaMaestra(2, 2, 2)
        for n, valor in enumerate((0, 1, 6, 11, 28, 37, 58, 71, 120, 137, 174, 
                                   195, 260, 285, 338, 367, 496, 529, 598, 635)):
            self.assertEqual(r[n], valor)

    def test_operador_getitem_1201(self):        

        r = RecurrenciaMaestra(1, 2, 0, 1) 
        for n, valor in enumerate((1, 2, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 
                                   5, 6, 6, 6, 6)):
            self.assertEqual(r[n], valor)

    def test_operador_getitem_431(self):        
        
        r = RecurrenciaMaestra(4, 3, 1)
        for n, valor in enumerate((0, 1, 2, 7, 8, 9, 14, 15, 16, 37, 38, 39, 44,
                                   45, 46, 51, 52, 53, 74, 75)):
            self.assertEqual(r[n], valor)
            
    # Casos de prueba para la generación sobre RecurrenciaMaestra.
    
    def comprueba_generacion(self, recurrencia, valores):
        it = iter(recurrencia)
        for v in valores:
            self.assertEqual(v, next(it))
            
    def test_generacion_222(self):
        self.comprueba_generacion(
            RecurrenciaMaestra(2, 2, 2), 
            (0, 1, 6, 11, 28, 37, 58, 71, 120, 137, 174, 195, 260, 285, 338, 
             367, 496, 529, 598, 635))
            
    def test_generacion_1201(self):            
        self.comprueba_generacion(
            RecurrenciaMaestra(1, 2, 0, 1), 
            (1, 2, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6))

    def test_generacion_431(self):            
        self.comprueba_generacion(
            RecurrenciaMaestra(4, 3, 1),
            (0, 1, 2, 7, 8, 9, 14, 15, 16, 37, 38, 39, 44, 45, 46, 51, 52, 53,
             74, 75))

## Ejecución de tests

In [7]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

........EEEFEEEFFEF
ERROR: test_generacion_1201 (__main__.TestRecurrenciaMaestra)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_7448/1850443133.py", line 73, in test_generacion_1201
    self.comprueba_generacion(
  File "/tmp/ipykernel_7448/1850443133.py", line 62, in comprueba_generacion
    it = iter(recurrencia)
TypeError: iter() returned non-iterator of type 'NoneType'

ERROR: test_generacion_222 (__main__.TestRecurrenciaMaestra)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_7448/1850443133.py", line 67, in test_generacion_222
    self.comprueba_generacion(
  File "/tmp/ipykernel_7448/1850443133.py", line 62, in comprueba_generacion
    it = iter(recurrencia)
TypeError: iter() returned non-iterator of type 'NoneType'

ERROR: test_generacion_431 (__main__.TestRecurrenciaMaestra)
-----------------------------------------